In [119]:
import os
import pandas as pd
#import re
#import numpy as np
from datetime import datetime
from openpyxl import load_workbook
import warnings
import pathlib 
#from pathlib import Path 
#import threading
from tqdm import tqdm
from datetime import datetime

warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')

# MANUAL input to change
your_ing_id = "PY40DL"
date = datetime.strptime('2024', "%Y")
edition = "August 2025"
combine_intermediate_files = True

# DO NOT CHANGE ANYTHING BELOW - only when developing code ##########################################################################
pap_categories = ["New financial product/-service/channel (PAP)", "Termination of financial product/-service/channel (PAP)", "Significant change financial product/-service/channel (PAP)"]
non_pap_categories = ["Risk assessment (non-PAP)", "Other change (non-PAP)"]


# determining operating system
is_windows = os.name == 'nt'
if is_windows:
    base_dir = os.path.join("C:", "\\Users", your_ing_id, "ING")
else:
    base_dir = pathlib.Path(f"/Users/{your_ing_id}/Library/CloudStorage/OneDrive-SharedLibraries-ING")

# Define paths dynamically
directory_path0 = os.getcwd()
directory_path = os.path.join(base_dir, "Product Evaluation and Risk Assessment Library (PEARL) - PEARL_Repository")
directory_path2 = os.path.join(base_dir, "Product Evaluation and Risk Assessment Library (PEARL) - MI Dashboard")
directory_path3 = os.path.join(directory_path0, "Intermediate results", edition)
directory_path4 = os.path.join(directory_path0, "Intermediate results combined", edition)
directory_path5 = os.path.join(directory_path0, "Final overview")
dir_output = os.path.join(os.getcwd(), "Output", edition)


if not os.path.exists(directory_path4):
    os.makedirs(directory_path4)

if not os.path.exists(dir_output):
    os.makedirs(dir_output)


final_overview = pd.read_csv(os.path.join(directory_path5, 'final_overview ' + edition +'.csv'), sep=';')
final_overview["Start Date"] = pd.to_datetime(final_overview["Start Date"])  # Convert to datetime
final_overview["End Date"] = pd.to_datetime(final_overview["End Date"])  # Convert to datetime

if is_windows:
    final_overview['Folder'] = final_overview['Folder'].str.replace('/', '\\')


# Print paths
print(f"Operating System: {'Windows' if is_windows else 'Mac/Linux'}")
print(f"PEARL Repository Path: {directory_path}")
print(f"MI Dashboard Path: {directory_path2}")
print(f"Intermediate Results Path: {directory_path3}")

Operating System: Windows
PEARL Repository Path: C:\Users\PY40DL\ING\Product Evaluation and Risk Assessment Library (PEARL) - PEARL_Repository
MI Dashboard Path: C:\Users\PY40DL\ING\Product Evaluation and Risk Assessment Library (PEARL) - MI Dashboard
Intermediate Results Path: c:\Users\PY40DL\OneDrive - ING\Projects\Risk Assesments\Intermediate results\August 2025


In [ ]:
# functions

def drop_duplicate_RA(df):

    df['File date'] = pd.to_datetime(df['File date'])
    df = df.sort_values(by='File date', ascending=False)

    latest_dates = df.groupby('File')['File date'].max().reset_index()

    df = df.merge(latest_dates, on=['File', 'File date'])

    df.reset_index(drop=True, inplace=True)
    return(df)


    
# create categories
def categorize_days(x):
    if x < 30:
        return "<30 days"
    elif x < 60:
        return "30-60 days"
    elif x < 90:
        return "60-90 days"
    elif x < 120:
        return "90-120 days"
    elif x < 180:
        return "120-180 days"
    else:
        return ">180 days"
    

def assigning_end_quarters(dt):
    non_missing_end_date = ~dt["End Date"].isna()
    dt["End Date"] = pd.to_datetime(dt["End Date"])  # Convert to datetime
    dt["Year"] = dt["End Date"].dt.year
    dt.loc[non_missing_end_date, "Quarter"] = dt["End Date"].dt.to_period("Q")
    #dt["Quarter"] = dt["Quarter"].fillna("Unknown").astype(str)
    #dt["Quarter"] = dt["End Date"].dt.to_period("Q").astype(str)

    cond = dt['Status'].isin(['5. Journey afgerond', '5. Journey finished', '5. Journey Cancelled', '5. Journey Gone live without approval', '6. Historical upload'])

    dt.loc[cond & non_missing_end_date, "Quarter_end"] = dt.loc[cond & non_missing_end_date, "Quarter"].astype(str)
    dt.loc[~cond, "Quarter_end"] = "Open"

    return(dt)


def quality_check(dt):

    # checking if file was read
    cond = dt['File'].isna()
    dt.loc[cond, "Missing data"] = True
    dt.loc[~cond, "Missing data"] = False

    # checking if the correct line is read
    cond = (dt['ORM_check'] == 'ORM') & (dt['IRM_check'] == 'IRM')
    dt.loc[cond, "Data correct"] = True  
    dt.loc[~cond, "Data correct"] = False 

    return(dt)


def structuring_data(dt, col, value, yes, no):
    cond = (dt[col] == value)
    dt.loc[cond, col] = yes
    dt.loc[~cond, col] = no

    dt[col] = pd.to_numeric(dt[col], errors='coerce')

    return(dt)


In [69]:

if combine_intermediate_files:
    # Initialize DataFrames
    dt1, dt2, dt3, dt4, dt5, dt6, dt7 = [pd.DataFrame() for _ in range(7)]

    # Check if directory is empty
    all_files = os.listdir(directory_path3)
    if not all_files:
        raise ValueError("Error: directory_path3 is empty!")

    # Process files
    for f in tqdm(all_files):
        file_path = os.path.join(directory_path3, f)

        # Ensure it's a file, not a directory
        if os.path.isfile(file_path):
            try:
                dt_f = pd.read_csv(file_path, sep=';', encoding='utf-8')
                if 'process_module_selection' in f:
                    dt1 = pd.concat([dt1, dt_f])
                elif 'risk_summary_approval' in f:
                    dt2 = pd.concat([dt2, dt_f])
                elif 'general_risk_ident_1' in f:
                    dt3 = pd.concat([dt3, dt_f])
                elif 'general_risk_ident_2' in f:
                    dt4 = pd.concat([dt4, dt_f])
                elif 'module_selected' in f:
                    dt5 = pd.concat([dt5, dt_f])
                elif 'assesment_stage' in f:
                    dt6 = pd.concat([dt6, dt_f])
                elif 'journey_summary' in f:
                    dt7 = pd.concat([dt7, dt_f])

            except Exception as e:
                print(f"Error processing file {f}: {e}")


    #drop duplicates
    dt1 = drop_duplicate_RA(dt1)
    dt2 = drop_duplicate_RA(dt2)
    dt3 = drop_duplicate_RA(dt3)
    dt4 = drop_duplicate_RA(dt4)
    dt5 = drop_duplicate_RA(dt5)
    #dt6 = drop_duplicate_RA(dt6)
    #dt7 = drop_duplicate_RA(dt7)

    # Create backup copies before modification
    dataframes = [dt1, dt2, dt3, dt4, dt5, dt6, dt7]
    dataframes_backup = [df.copy() for df in dataframes]

# Normalize folder paths (Mac-safe)
#for i in range(len(dataframes)):
 #   if 'Folder' in dataframes[i].columns:
  #      dataframes[i]['Folder'] = dataframes[i]['Folder'].apply(lambda x: str(Path(x)))


    # merge with final overview each 
    dt1 = dt1.merge(final_overview, on="Folder", how="right")
    dt2 = dt2.merge(final_overview, on="Folder", how="right")
    dt3 = dt3.merge(final_overview, on="Folder", how="right")
    dt4 = dt4.merge(final_overview, on="Folder", how="right")
    dt5 = dt5.merge(final_overview, on="Folder", how="right")

    dt1.to_csv(os.path.join(directory_path4, 'dt1.csv'), sep = ';')
    dt2.to_csv(os.path.join(directory_path4, 'dt2.csv'), sep = ';')
    dt3.to_csv(os.path.join(directory_path4, 'dt3.csv'), sep = ';')
    dt4.to_csv(os.path.join(directory_path4, 'dt4.csv'), sep = ';')
    dt5.to_csv(os.path.join(directory_path4, 'dt5.csv'), sep = ';')

else:
    dt1 = pd.read_csv(os.path.join(directory_path4, 'dt1.csv'), sep = ';')
    dt2 = pd.read_csv(os.path.join(directory_path4, 'dt2.csv'), sep = ';')
    dt3 = pd.read_csv(os.path.join(directory_path4, 'dt3.csv'), sep = ';')
    dt4 = pd.read_csv(os.path.join(directory_path4, 'dt4.csv'), sep = ';')
    dt5 = pd.read_csv(os.path.join(directory_path4, 'dt5.csv'), sep = ';')

100%|██████████| 9173/9173 [02:05<00:00, 72.97it/s]


In [70]:
check = final_overview[final_overview['Process Category'].isin(pap_categories)]

check2 = final_overview[final_overview['ID'] == '498']

In [71]:
### data manipulation for each individual df process_module_selection
dt1 = quality_check(dt1)

dt1 = structuring_data(dt1, 'IRM', 'Yes', '1', '0')
dt1 = structuring_data(dt1, 'ORM', 'Yes', '1', '0')

In [72]:
# IRM & ORM dt2 risk summary approval 
dt2 = quality_check(dt2)

dt2 = structuring_data(dt2, 'ORM_invited_or_challange', 'To be invited for challenge', '1', '0')
dt2 = structuring_data(dt2, 'IRM_invited_or_challange', 'To be invited for challenge', '1', '0')

In [73]:
# general_risk_ident_1
dt3 = quality_check(dt3)

dt3 = structuring_data(dt3, 'ORM_invited', 'Yes', '1', '0')
dt3 = structuring_data(dt3, 'IRM_invited', 'Yes', '1', '0')

dt3 = structuring_data(dt3, 'ORM_part_of_risk_asses', 'Yes', '1', '0')
dt3 = structuring_data(dt3, 'IRM_part_of_risk_asses', 'Yes', '1', '0')


In [74]:
# general_risk_ident_2
dt4 = quality_check(dt4)

dt4 = structuring_data(dt4, 'ORM_person', 'Not applicable', '0', '1')
dt4 = structuring_data(dt4, 'IRM_person', 'Not applicable', '0', '1')

dt4 = structuring_data(dt4, 'ORM_opinion', 'Not applicable', '0', '1')
dt4 = structuring_data(dt4, 'IRM_opinion', 'Not applicable', '0', '1')

dt4 = structuring_data(dt4, 'ORM_challenge', 'Not applicable', '0', '1')
dt4 = structuring_data(dt4, 'IRM_challenge', 'Not applicable', '0', '1')


In [75]:
### PAP output same output, but filtered on non PAP
dt1 = assigning_end_quarters(dt1)
dt1_PAP = dt1[dt1["Process Category"].isin(pap_categories)]
#filtered_data_PAP

dt1_grouped_PAP = dt1_PAP.groupby(['Missing data', 
                                   'Data correct', 
                                   'Type', 
                                   'Quarter_end']).agg({'ORM': 'sum', 
                                                        'IRM': 'sum', 
                                                        'Folder': 'count'})

dt1_grouped_PAP.to_csv(os.path.join(dir_output, "PAP_involvment.csv"), sep = ";")

C:\Users\PY40DL\AppData\Local\Temp\ipykernel_279260\2988048171.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Open' has dtype incompatible with period[Q-DEC], please explicitly cast to a compatible dtype first.
  dt.loc[~cond, "Quarter_end"] = "Open"


In [76]:
### pivot for overall dt1 output 
dt1_non_PAP = dt1[dt1["Process Category"].isin(non_pap_categories)]

dt1_grouped_non_PAP = dt1_non_PAP.groupby(['Missing data', 
                                           'Data correct', 
                                           'Type', 
                                           'Quarter_end']).agg({'ORM': 'sum', 
                                                                'IRM': 'sum', 
                                                                'Folder': 'count'})

dt1_grouped_non_PAP.to_csv(os.path.join(dir_output, "Change_RA_involvment.csv"), sep = ";")

In [77]:
# create percentages
dt1_non_PAP_percent = dt1_grouped_non_PAP.copy()

# Divide ORM & IRM by total Folder count per quarter
cols_to_percent = ["ORM", "IRM"]
dt1_non_PAP_percent[cols_to_percent] = dt1_non_PAP_percent[cols_to_percent].div(
    dt1_non_PAP_percent["Folder"], axis=0) * 100
dt1_non_PAP_percent[cols_to_percent] = dt1_non_PAP_percent[cols_to_percent].round(2)

dt1_non_PAP_percent.to_csv(os.path.join(dir_output, "Change_RA_involvment_perc.csv"), sep = ";")

In [78]:
### first data analysis for dt2 to report # invited_challenges
dt2 = assigning_end_quarters(dt2)

#dt2_non_PAP = dt2[dt2["Process Category"].isin(non_pap_categories)]

dt2_grouped = dt2.groupby(['Missing data', 
                           'Data correct', 
                           'Quarter_end']).agg({'ORM_invited_or_challange': 'sum', 
                                       'IRM_invited_or_challange': 'sum', 
                                       'Folder': 'count'})

# create percentages
dt2_percent = dt2_grouped.copy()

# Divide ORM & IRM by total Folder count per quarter
cols_to_percent = ["ORM_invited_or_challange", "IRM_invited_or_challange"]
dt2_percent[cols_to_percent] = dt2_percent[cols_to_percent].div(
    dt2_percent["Folder"], axis=0) * 100
dt2_percent[cols_to_percent] = dt2_percent[cols_to_percent].round(2)

C:\Users\PY40DL\AppData\Local\Temp\ipykernel_279260\2988048171.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Open' has dtype incompatible with period[Q-DEC], please explicitly cast to a compatible dtype first.
  dt.loc[~cond, "Quarter_end"] = "Open"


In [79]:
### first data analysis for dt3
dt3 = assigning_end_quarters(dt3)

dt3_non_PAP = dt3[dt3["Process Category"].isin(non_pap_categories)]


dt3_grouped_non_PAP = dt3_non_PAP.groupby(['Missing data', 
                           'Data correct', 
                           'Quarter_end']).agg({'ORM_invited': 'sum', 
                                         'IRM_invited': 'sum',    
                                         'ORM_part_of_risk_asses': 'sum', 
                                         'IRM_part_of_risk_asses': 'sum', 
                                         'Folder': 'count'})
dt3_grouped_non_PAP.to_csv(os.path.join(dir_output, "Change_RA_specific_involvment.csv"), sep = ";")

C:\Users\PY40DL\AppData\Local\Temp\ipykernel_279260\2988048171.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Open' has dtype incompatible with period[Q-DEC], please explicitly cast to a compatible dtype first.
  dt.loc[~cond, "Quarter_end"] = "Open"


In [80]:
# create percentages
dt3_percent_non_PAP = dt3_grouped_non_PAP.copy()

# Divide ORM & IRM by total Folder count per quarter
cols_to_percent = ["ORM_invited", "IRM_invited", "ORM_part_of_risk_asses", "IRM_part_of_risk_asses"]
dt3_percent_non_PAP[cols_to_percent] = dt3_percent_non_PAP[cols_to_percent].div(
    dt3_percent_non_PAP["Folder"], axis=0) * 100
dt3_percent_non_PAP[cols_to_percent] = dt3_percent_non_PAP[cols_to_percent].round(2)
dt3_percent_non_PAP.to_csv(os.path.join(dir_output, "Change_RA_specific_involvment_perc.csv"), sep = ";")

In [81]:
### first data analysis dt4
dt4 = assigning_end_quarters(dt4)

dt4_non_PAP = dt4[dt4["Process Category"].isin(non_pap_categories)]


dt4_grouped_non_PAP = dt4_non_PAP.groupby(['Missing data', 
                           'Data correct', 
                           'Quarter']).agg({'ORM_person': 'sum', 
                                         'IRM_person': 'sum',    
                                         'ORM_opinion': 'sum', 
                                         'IRM_opinion': 'sum',
                                         'ORM_challenge': 'sum', 
                                         'IRM_challenge': 'sum', 
                                         'Folder': 'count'})


C:\Users\PY40DL\AppData\Local\Temp\ipykernel_279260\2988048171.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Open' has dtype incompatible with period[Q-DEC], please explicitly cast to a compatible dtype first.
  dt.loc[~cond, "Quarter_end"] = "Open"


In [82]:
# create percentages
dt4_percent_non_PAP = dt4_grouped_non_PAP.copy()

# Divide ORM & IRM by total Folder count per quarter
cols_to_percent = ["ORM_person", "IRM_person", "ORM_opinion", "IRM_opinion", "ORM_challenge", "IRM_challenge"]
dt4_percent_non_PAP[cols_to_percent] = dt4_percent_non_PAP[cols_to_percent].div(
    dt4_percent_non_PAP["Folder"], axis=0) * 100
dt4_percent_non_PAP[cols_to_percent] = dt4_percent_non_PAP[cols_to_percent].round(2)

In [83]:
### module_selected

dt5 = assigning_end_quarters(dt5)


C:\Users\PY40DL\AppData\Local\Temp\ipykernel_279260\2988048171.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Open' has dtype incompatible with period[Q-DEC], please explicitly cast to a compatible dtype first.
  dt.loc[~cond, "Quarter_end"] = "Open"


In [88]:
# add column with sum # modules selected
df_5_module_selected =dt5.groupby(["Folder", 
                                    "File", 
                                    "Quarter_end"]).agg({"Applicable": lambda x: x.sum()  }).reset_index()

df_5_module_selected.rename(columns={"Applicable": "Applicable_modules_selected"}, inplace=True)

In [89]:
### first data analysis dt5 with # modules selected
df_wide = df_5_module_selected.pivot_table(index="Quarter_end", 
                                           columns="Applicable_modules_selected", 
                                           values="File", 
                                           aggfunc="count").fillna(0)

# Reset index to make it cleaner
df_wide.reset_index(inplace=True)
#df_wide

In [90]:
df_percent = df_wide.copy()  
module_cols = df_percent.columns.difference(["Quarter_end"])
df_percent[module_cols] = df_percent[module_cols].div(df_percent[module_cols].sum(axis=1), axis=0) * 100
df_percent[module_cols] = df_percent[module_cols].round(2)
#df_percent

In [ ]:
# table with duraction RA's per category Richards request

today_date = datetime.today().date()
final_overview = assigning_end_quarters(final_overview)
cond = final_overview['Quarter_end'].isna()
final_overview.loc[cond, 'Quarter_end'] = 'Status_End_Date_incorrect'
final_overview["End Date"] = final_overview["End Date"].fillna(pd.Timestamp(today_date))

# calculate duration
final_overview["Duration"] = (final_overview["End Date"] - final_overview["Start Date"]).dt.days
final_overview["Duration"] = pd.to_numeric(final_overview["Duration"], errors='coerce')

final_overview['Category'] = final_overview['Duration'].apply(categorize_days)



duration_analysis = final_overview.copy()

# excluding journeys on hold


duration_analysis = duration_analysis[duration_analysis["Process Category"].isin(non_pap_categories)] # WHY???????????????????????????????????

duration_analysis = duration_analysis.groupby(['Type','Category', 'Quarter_end']).agg({'Folder': 'count'})


category_order = ["<30 days", "30-60 days", "60-90 days", "90-120 days", "120-180 days", ">180 days"]
duration_analysis = duration_analysis.reset_index()
duration_analysis['Category'] = pd.Categorical(duration_analysis['Category'], categories=category_order, ordered=True)
duration_analysis = duration_analysis.sort_values(by='Category')
duration_analysis

,Type,Category,Quarter_end,Folder
44,Change,<30 days,Status_End_Date_incorrect,1
43,Change,<30 days,Open,31
42,Change,<30 days,2025Q3,11
41,Change,<30 days,2025Q2,10
40,Change,<30 days,2025Q1,19
39,Change,<30 days,2024Q4,12
38,Change,<30 days,2024Q3,7
37,Change,<30 days,2024Q2,18
36,Change,<30 days,2024Q1,20
17,Change,30-60 days,Status_End_Date_incorrect,3


In [122]:
## pivot duration categories * quarters
duration_analysis_pivoted = duration_analysis.pivot_table(
    index = 'Category',
    columns=[ 'Quarter_end'],
    values='Folder',  
    aggfunc = 'sum',
    fill_value=0  
)
duration_analysis_pivoted

C:\Users\PY40DL\AppData\Local\Temp\ipykernel_279260\3421939333.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  duration_analysis_pivoted = duration_analysis.pivot_table(


Quarter_end,2024Q1,2024Q2,2024Q3,2024Q4,2025Q1,2025Q2,2025Q3,Open,Status_End_Date_incorrect
Category,,,,,,,,,
<30 days,20,18,7,12,19,10,11,31,1
30-60 days,6,8,7,8,9,10,3,45,3
60-90 days,4,7,11,8,5,10,7,29,2
90-120 days,12,5,6,7,4,5,3,34,4
120-180 days,13,7,11,17,8,18,7,74,6
>180 days,34,38,27,56,34,46,18,243,92


In [126]:
## analysis duration in days average per tribe to see whether there are some tribes with more duration. Results do not show very clear differences

duration_analysis2 = final_overview[final_overview["Process Category"].isin(non_pap_categories)] # WHY????????????????????????????????????????

duration_analysis2 = duration_analysis2.groupby(['Tribe']).agg({'Duration': 'mean', 'Folder': 'count'}).reset_index()

In [ ]:
## trying to connect dt3 & dt4 to have the information from sheet risk_identification in one file
#merged_df = pd.merge(dt3, dt4, on=['Folder', 'File'], how='inner', suffixes=('', '_dup'))
#merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_dup')]

#merged_df
#merged_df2 = merged_df[merged_df["Process Category"].isin(non_pap_categories)]
#merged_df_pivot = merged_df2.groupby(['Missing data', 
#                           'Data correct', 
#                          # 'Type',
#                           'ORM_invited',
#                           'IRM_invited',
#                           'ORM_part_of_risk_asses',
#                           'IRM_part_of_risk_asses', 
#                           'ORM_person',
#                           'IRM_person',
#                           #'Quarter'
#                                     ]).agg({
#                                       'Folder': 'count'})

#merged_df_pivot

### does not really add up to something meaningful. 